In [1]:
from PIL import Image
# from scipy.misc import imresize,imsave
from scipy import misc


import pandas as pd
import os
from tqdm import tqdm_notebook as tqdm
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

PATH_TO_DATA="/Volumes/HDD/Datasets/coursework/vgg_face_dataset/images_50_crop/"
PATH_TO_OUT="/Volumes/HDD/Datasets/coursework/vgg_face_dataset/image_50_final/"  #Удялять плохие фото вручную нужно от сюда
PATH_TO_ARRAY="/Volumes/HDD/Datasets/coursework/vgg_face_dataset/image_50_array/"
PATH_TO_LABELS="/Volumes/HDD/Datasets/coursework/vgg_face_dataset/files/_all_in_one_50.csv"

### Вычисляю оптимальный размер к которому буду приводить

In [3]:
size=[]
for file in tqdm(os.listdir(PATH_TO_DATA)):
        name,ext=os.path.splitext(file)
        if ext==".jpg":
            img = Image.open(PATH_TO_DATA+file)
            size.append(img.size)

HBox(children=(IntProgress(value=0, max=16705), HTML(value='')))

In [4]:
tmp={}
for key,val in zip(Counter(size).keys(),Counter(size).values()):
    tmp[val]=key

In [5]:
tmp

{1: (286, 286),
 2: (232, 232),
 3: (384, 384),
 4: (54, 54),
 5: (133, 134),
 6: (501, 501),
 7: (42, 42),
 8: (52, 51),
 9: (47, 47),
 10: (435, 435),
 11: (47, 48),
 13: (290, 291),
 14: (436, 436),
 15: (254, 253),
 16: (406, 407),
 17: (125, 125),
 18: (334, 334),
 19: (134, 133),
 20: (90, 89),
 21: (436, 435),
 22: (330, 329),
 23: (58, 58),
 24: (146, 146),
 25: (124, 124),
 26: (109, 108),
 28: (77, 76),
 29: (178, 179),
 30: (67, 67),
 31: (268, 267),
 32: (58, 59),
 33: (82, 81),
 34: (81, 82),
 35: (191, 191),
 36: (177, 177),
 38: (406, 406),
 39: (84, 84),
 41: (467, 467),
 42: (153, 154),
 44: (48, 48),
 46: (95, 95),
 48: (88, 87),
 49: (205, 206),
 51: (73, 72),
 52: (78, 77),
 53: (101, 100),
 54: (68, 68),
 55: (77, 78),
 56: (179, 178),
 58: (221, 221),
 62: (155, 155),
 63: (218, 218),
 65: (146, 145),
 66: (271, 271),
 67: (76, 76),
 68: (221, 220),
 70: (220, 221),
 71: (220, 220),
 73: (307, 308),
 76: (83, 84),
 78: (176, 177),
 81: (102, 102),
 82: (191, 192),

### Привожу к одному размеру

In [2]:
def resized(path_to_data,path_to_out,size):
    for file in tqdm(os.listdir(path_to_data)):
            name,ext=os.path.splitext(file)
            if ext==".jpg":
                img = Image.open(path_to_data+file)
                img=misc.imresize(img,size)
                misc.imsave(path_to_out+file,img)

In [ ]:
size=(128,128)
resized(PATH_TO_DATA,PATH_TO_OUT,size)

### Преобразую картинку в массив numpy и удаляю тех людей для которых меньше n фото

In [2]:
def image_to_array(path_to_images,path_to_labels,bad_people=[]):
    num=sum([1 for image in os.listdir(path_to_images) if os.path.splitext(image)[1]==".jpg"])
    labels=pd.read_csv(PATH_TO_LABELS)[["0","num_image"]].values
    
    
    i=0
    X=np.zeros(shape=(num,128,128,3),dtype="int32")
    y=np.zeros(shape=(num),dtype=object)
    
    for image in tqdm(os.listdir(path_to_images)):
        name,ext=os.path.splitext(image)
        if ext==".jpg":
            img = misc.imread(path_to_images+image,flatten=False)
            name_people=labels[labels[:,1]==int(name)][0][0]
            if img.shape==(128, 128,3) and name_people not in bad_people:
                X[i] = np.asarray( img, dtype="int32")
                y[i]=name_people
                i+=1
            else:
                y[i]="NaN"
                i+=1
        
    X=X[y!='NaN']
    y=y[y!='NaN']
    return X,y

In [8]:
def people_to_delete(n,y):
    bad_people=[]
    for key,val in zip(Counter(y).keys(),Counter(y).values()):
        if val<=n:
            bad_people.append(key)
    return bad_people

In [9]:
X,y=image_to_array(PATH_TO_OUT,PATH_TO_LABELS)

In [10]:
#Удаляю людей у которых мало фото
bad_people=people_to_delete(3,y)

X,y=image_to_array(PATH_TO_OUT,PATH_TO_LABELS,bad_people=bad_people)

### Сохранение X и y для дальнейшего использования

In [11]:
%%time
np.save(PATH_TO_ARRAY+"X_data",X)
np.save(PATH_TO_ARRAY+"y_data",y)

CPU times: user 671 ms, sys: 12.1 s, total: 12.8 s
Wall time: 56.9 s
